In [0]:
# Setup :)
import numpy as np
import time
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.enable_eager_execution(config=config)

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Conv3D, Dense, Conv1D, Input, Reshape, Conv3DTranspose, BatchNormalization
from tensorflow.keras.losses import logcosh
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow import GradientTape
from tensorflow.train import Saver
from tensorflow import Session
from tensorflow.train import AdamOptimizer

# Use Adam optimizer.
optimizer = AdamOptimizer(1e-4)

In [17]:
# Creates autoencoder 
########################## ENCODER NETWORK #################################
# Input is 32x32x32 point cloud
voxel_input = Input(shape=(32, 32, 32, 1))

# First convolutional layer: outputs 30x30x30
encode_c1 = Conv3D(8, kernel_size=3, activation='elu', padding='valid',
              data_format='channels_last', kernel_regularizer=l2(l=0.01))(voxel_input)
encode_b1 = BatchNormalization()(encode_c1)

# Second convolutional layer: outputs 15x15x15 (downsamples via striding)
encode_c2 = Conv3D(16, kernel_size=3, activation='elu', padding='same',
              strides=(2, 2, 2), kernel_regularizer=l2(l=0.01))(encode_b1)
encode_b2 = BatchNormalization()(encode_c2)

# Third convolutional layer: outputs 13x13x13
encode_c3 = Conv3D(32, kernel_size=3, activation='elu', padding='valid',
                  kernel_regularizer=l2(l=0.01))(encode_b2)
encode_b3 = BatchNormalization()(encode_c3)

# Fourth convolutional layer: outputs 7x7x7 (downsamples via striding)
encode_c4 = Conv3D(64, kernel_size=3, activation='elu', padding='same',
              strides=(2, 2, 2), kernel_regularizer=l2(l=0.01))(encode_b3)
encode_b4 = BatchNormalization()(encode_c4)

# Fifth layer, fully connected: outputs 343
encode_f5 = Flatten()(encode_b4)
encode_b5 = BatchNormalization()(encode_f5)

# Sixth layer - LATENT LAYER: outputs 100
latent = Dense(100, use_bias=True, activation='elu',
               kernel_regularizer=l2(l=0.01))(encode_b5)

encoder = Model(inputs=voxel_input, outputs=latent)

# Run this to get summary of encoder's layers
# encoder.summary()


########################## DECODER NETWORK #################################
# Latent space of 1D dimension 100 is input for decoder
decoder_input = Input(shape=(100,))

# First layer of decoder, fully connected layer: outputs 343
decode_f1 = Dense(343, use_bias=True, activation='elu',
                 kernel_regularizer=l2(l=0.01))(decoder_input)
decode_b1 = BatchNormalization()(decode_f1)

# Reshape layer from fully connected to 7x7x7
# must add spacial dimension for convolutions to work
decode_reshape = Reshape((7, 7, 7, 1), input_shape=(343,))(decode_b1)

# Second convolutional layer: convolutes fully connected layer into 7x7x7
decode_c2 = Conv3D(64, kernel_size=3, activation='elu',
                   padding='same', kernel_regularizer=l2(l=0.01))(decode_reshape)
decode_b2 = BatchNormalization()(decode_c2)

# Third layer (second convolutional layer): outputs 15x15x15
decode_c3 = Conv3DTranspose(32, kernel_size=3, activation='elu',padding='valid',
                   strides=(2, 2, 2), kernel_regularizer=l2(l=0.01))(decode_b2)
decode_b3 = BatchNormalization()(decode_c3)

# Fourth convolutional layer: outputs 15x15x15
decode_c4 = Conv3DTranspose(16, kernel_size=3, activation='elu',
                   padding='same', kernel_regularizer=l2(l=0.01))(decode_b3)
decode_b4 = BatchNormalization()(decode_c4)

# Fifth convolutional layer: outputs 32x32x32
decode_c5 = Conv3DTranspose(8, kernel_size=3, activation='elu', padding='valid',
                   strides=(2, 2, 2), output_padding=1, kernel_regularizer=l2(l=0.01))(decode_b4)
decode_b5 = BatchNormalization()(decode_c5)

# OUTPUT LAYERRRRRRRR!!! Sigmoid function to output probability each voxel is filled
decode_output = Conv3DTranspose(1, kernel_size=3, activation='sigmoid',
                   padding='same', kernel_regularizer=l2(l=0.01))(decode_b5)

decoder = Model(inputs=decoder_input, outputs=decode_output)

# Run this to get layer summary for decode_reshape
# decoder.summary()

################################# AUTOENCODER ###############################
reconstruction = decoder(encoder(voxel_input))
ae = Model(inputs=voxel_input, outputs=reconstruction)

ae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 32, 1)     0         
_________________________________________________________________
model_6 (Model)              (None, 100)               2356500   
_________________________________________________________________
model_7 (Model)              (None, 32, 32, 32, 1)     111136    
Total params: 2,467,636
Trainable params: 2,422,566
Non-trainable params: 45,070
_________________________________________________________________


In [21]:
# Load already trained weights 
model_checkpoint_file = 'ae_checkpoint'
ae.load_weights(model_checkpoint_file)

# Get Decoder portion of AE
decoder = ae.layers[2]
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 343)               34643     
_________________________________________________________________
batch_normalization_v1_25 (B (None, 343)               1372      
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 7, 1)        0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 7, 7, 7, 64)       1792      
_________________________________________________________________
batch_normalization_v1_26 (B (None, 7, 7, 7, 64)       256       
_________________________________________________________________
conv3d_transpose_8 (Conv3DTr (None, 15, 15, 15, 32)    55328     
__________

In [0]:
################### MODIFIED BINARY CROSS ENTROPY LOSS FUNCTION ############
# Binary cross entropy but with value clamping
def lambda_binary_crossentropy(y_true, y_pred):
  # Only want loss from the voxels that are present in the partial
#   y_pred = tf.math.multiply(y_true, y_pred)
  
  y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
  binary_entr = y_true*((-y_true * tf.log(y_pred)) - ((1.0-y_true) * tf.log(1.0-y_pred)))

  # getting tensor values into scalar
  loss = tf.reduce_sum(binary_entr, axis=1)
  scalar_loss = tf.reduce_mean(loss)

  return scalar_loss

In [0]:
# Gradient computation with given model, input latent space, and partial voxel
def compute_gradients(model, latent, partial):
#   input_tensor = tf.cast(x[0], dtype=tf.float32)
  with GradientTape() as tape:
    tape.watch(latent)
    y = model(latent)
    loss = lambda_binary_crossentropy(partial, y)
    
    # model.trainable_variables
    #????? loss is scalar? how is it the gradient target? 
    return tape.gradient(loss, latent), loss
 
# 
# "Generates" a latent space by passing an initially random latent vector
# through the decoder netowrk, and computing loss between 
# decoder's output and a partial view voxel.
#
# Runs updates until loss is under loss_thresh
#
def recover_latent(partial, loss_thresh):
  # Latent vector starts as a random vector
  curr_latent = tf.random.uniform(shape=(1, 100))
  curr_latent = tf.Variable(curr_latent)
  
  loss = float('inf')
  iter = 0
  while loss > loss_thresh:
    print 'Iteration: ', str(iter)
    # Compute gradient and loss of current latent vector
    gradients, loss = compute_gradients(decoder, curr_latent, partial)
    print loss
    gradients = tf.reshape(gradients, (1, 100))
    
    # Updating latent vector according to gradient
    optimizer.apply_gradients(zip([gradients], [curr_latent])) 
    iter += 1

In [92]:
# Testing on dummy voxel 
dummy_vox = tf.random.uniform(shape=(32, 32, 32))

recover_latent(dummy_vox, 10.0)

Iteration:  0
tf.Tensor(160.90533, shape=(), dtype=float32)
Iteration:  1
tf.Tensor(160.90417, shape=(), dtype=float32)
Iteration:  2
tf.Tensor(160.90233, shape=(), dtype=float32)
Iteration:  3
tf.Tensor(160.9013, shape=(), dtype=float32)
Iteration:  4
tf.Tensor(160.90292, shape=(), dtype=float32)
Iteration:  5
tf.Tensor(160.8974, shape=(), dtype=float32)
Iteration:  6
tf.Tensor(160.89789, shape=(), dtype=float32)
Iteration:  7
tf.Tensor(160.89932, shape=(), dtype=float32)
Iteration:  8
tf.Tensor(160.89276, shape=(), dtype=float32)
Iteration:  9
tf.Tensor(160.89258, shape=(), dtype=float32)
Iteration:  10
tf.Tensor(160.89026, shape=(), dtype=float32)
Iteration:  11
tf.Tensor(160.88947, shape=(), dtype=float32)
Iteration:  12
tf.Tensor(160.87856, shape=(), dtype=float32)
Iteration:  13
tf.Tensor(160.88065, shape=(), dtype=float32)
Iteration:  14
tf.Tensor(160.8805, shape=(), dtype=float32)
Iteration:  15
tf.Tensor(160.88354, shape=(), dtype=float32)
Iteration:  16
tf.Tensor(160.87442, s

KeyboardInterrupt: ignored